# Intro

This notebook is to show how to load behavioral and neural data, and to show how to transform the neural data into trial-by-trial datasets

---
# Setup

In [10]:
##% imports
from popy.io_tools import load_behavior, load_neural_data

from popy.neural_data_tools import build_trial_dataset, merge_behavior, downsample_time, add_firing_rates

---

In [11]:
monkey, session = 'ka', '210322'

# Behavior

In [12]:
##% load data
# load behavior data
behavior = load_behavior(monkey, session)
behavior = behavior.dropna()

behavior.head()

,monkey,session,trial_id,trial_start_time,lever_touch_time,lever_validation_time,lever_release_time,target_touch_time,target_validation_time,feedback_time,trial_end_time,block_id,best_target,target,feedback
16734,ka,210322,0,9.7919,51.9560,52.9768,53.4187,54.5254,55.0988,55.6184,59.6561,0,1,2.0,0.0
16735,ka,210322,1,59.6561,59.7454,60.7601,61.2780,61.8468,62.3944,62.9092,67.6624,0,1,3.0,0.0
16736,ka,210322,2,67.6624,68.2720,69.2893,69.8931,70.4462,70.9757,71.4914,76.0635,0,1,1.0,1.0
16737,ka,210322,3,76.0635,78.3176,79.3384,79.7376,80.0704,80.6119,81.1261,85.1502,0,1,1.0,0.0
16738,ka,210322,4,85.1502,85.9045,86.9218,87.2708,87.6534,88.1539,88.6702,92.9438,0,1,3.0,0.0


# Neural data

In [13]:
##% load neural data
# load neural data
neural_data = load_neural_data(monkey, session, hz=1000)
neural_data

<xarray.Dataset>
Dimensions:           (unit: 47, time: 3273607)
Coordinates:
  * unit              (unit) object 'LPFC_03_03' 'LPFC_03_01' ... 'MCC_16_01'
  * time              (time) float64 7.792 7.793 7.794 ... 3.281e+03 3.281e+03
    unit_id_original  (unit) int64 67 65 66 8 68 9 16 ... 48 49 53 52 61 59 58
    channel           (unit) int64 3 3 3 4 4 4 5 5 6 ... 12 12 13 13 14 16 16 16
    area              (unit) object 'LPFC' 'LPFC' 'LPFC' ... 'MCC' 'MCC' 'MCC'
    trial_id          (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    epoch_id          (time) float64 nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    spike_trains      (unit, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    bin_size:  0.001

In [5]:
neural_data = add_firing_rates(neural_data, method='gauss', std=0.05)

neural_data

KeyboardInterrupt: 

In [ ]:
neural_data = downsample_time(neural_data, sr_new=100)

neural_data

<xarray.Dataset>
Dimensions:           (unit: 47, time: 327359)
Coordinates:
  * unit              (unit) object 'LPFC_03_03' 'LPFC_03_01' ... 'MCC_16_01'
  * time              (time) float64 7.8 7.81 7.82 ... 3.281e+03 3.281e+03
    unit_id_original  (unit) int64 67 65 66 8 68 9 16 ... 48 49 53 52 61 59 58
    channel           (unit) int64 3 3 3 4 4 4 5 5 6 ... 12 12 13 13 14 16 16 16
    area              (unit) object 'LPFC' 'LPFC' 'LPFC' ... 'MCC' 'MCC' 'MCC'
    trial_id          (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    epoch_id          (time) float64 nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    spike_trains      (unit, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    firing_rates      (unit, time) float32 0.0 0.0 0.0 0.0 ... 18.67 18.3 18.09
Attributes:
    bin_size:  0.01

# Build trial-by-trial datasets

In [ ]:
# build trial dataset and merge with behavior
neural_dataset = build_trial_dataset(neural_data, mode='centering', center_on_epoch_start=5, center_window=(-3, 3))
neural_dataset = merge_behavior(neural_dataset, behavior)

neural_dataset

<xarray.Dataset>
Dimensions:                 (unit: 47, trial_id: 425, time: 600)
Coordinates: (12/21)
  * unit                    (unit) object 'LPFC_03_03' ... 'MCC_16_01'
    unit_id_original        (unit) int64 67 65 66 8 68 9 ... 49 53 52 61 59 58
    channel                 (unit) int64 3 3 3 4 4 4 5 ... 12 13 13 14 16 16 16
    area                    (unit) object 'LPFC' 'LPFC' 'LPFC' ... 'MCC' 'MCC'
  * trial_id                (trial_id) int64 0 1 2 3 4 5 ... 420 421 422 423 424
  * time                    (time) float64 -3.0 -2.99 -2.98 ... 2.97 2.98 2.99
    ...                      ...
    feedback_time           (trial_id) float64 55.62 62.91 ... 3.276e+03
    trial_end_time          (trial_id) float64 59.66 67.66 ... 3.279e+03
    block_id                (trial_id) int64 0 0 0 0 0 0 0 0 ... 9 9 9 9 9 9 9 9
    best_target             (trial_id) int64 1 1 1 1 1 1 1 1 ... 3 3 3 3 3 3 3 3
    target                  (trial_id) float64 2.0 3.0 1.0 1.0 ... 3.0 3.0 2.0
    feedback                (trial_id) float64 0.0 0.0 1.0 0.0 ... 0.0 0.0 1.0
Data variables:
    spike_trains            (trial_id, unit, time) float64 0.0 0.0 ... 0.0 0.0
    firing_rates            (trial_id, unit, time) float32 0.0 0.0 ... 0.1923
Attributes:
    bin_size:  0.01